#### Objective: 
To use the sensor signals from the CNC machine to detect if a cutting tool being used is worn out. Theoretically this allows the CNC operator to detect in real time that the cutting tool is worn out in order to take corrective action prior to the resultant part ending in a quality failure.

#### Approach: 
break the problem into parts:

* the ability to detect that the tool being used is worn out
* determining if there is a correlation between a worn tool and a failed part
* the ability to detect if tool wear is leading to a bad part or is within acceptable ranges.

#### Hypothesis:

* within the sensor readings from the CNC machine there is enough signal to detect that a tool is wearing out
* if a correlation exists between a worn tool and a failed part, there is signal to detect whether the tool wear is enough to lead to a failed part

In [1]:
from pprint import pprint

import pandas as pd
import numpy as np

# Standard plotly imports
import plotly.graph_objects as go

fig = go.Figure()
import plotly.figure_factory as pff
from plotly.subplots import make_subplots

import cufflinks
cufflinks.go_offline(connected=True)

import matplotlib.pyplot as plt

from sklearn.metrics import matthews_corrcoef
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA

In [2]:
#!pip install kaggle
#!pip install shap
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 67 bytes


#### **Objective:**
Since there is contradictory language in the description of the dataset I'm not sure that I can use this dataset as I initially desired. However, I will apply the same techinique that I was going to use to detect anomalies within the same timeseries dataset to validate my assumptions about the data itself.

#### **Contradictory language:**
"Outputs per experiment include tool condition (unworn and worn tools) and whether or not the tool passed visual inspection."
vs
"...could be performed for identification of worn and unworn cutting tools. Eight experiments were run with an unworn tool while ten were run with a worn tool (see tool_condition column..." & "...could be used to detect when a workpiece is not being held in the vise with sufficient pressure to pass visual inspection (see passed_visual_inspection column for indication of visual flaws)."

This language leaves me to question whether the worn tool experiments were started with a worn tool or started with an unworn tool and resulted in a worn tool at the end. My assumption is that the full experiment was run with either a worn or unworn tool from the beginning of that experimental run. I can validate this by applying the Mahalanobis distance calculation to an unworn tool run (or to multiple unworn tool runs) to find a threshold. Then use this threshold to detect at which point in the worn tool runs the experiment exceeds the threshold. If the threshold is exceed at or very close to the beginning of the experimental run, then the run was likely started with a worn tool. However, if the threshold is not exceeded until late in the experimental run, it is likely that the tool started out unworn and became worn by the end of the experimental run.

In [3]:
!kaggle datasets download 'shasun/tool-wear-detection-in-cnc-mill/data'

  0% 0.00/2.57M [00:00<?, ?B/s]
100% 2.57M/2.57M [00:00<00:00, 65.5MB/s]


In [4]:
!unzip tool-wear-detection-in-cnc-mill.zip

Archive:  tool-wear-detection-in-cnc-mill.zip
  inflating: README.txt              
  inflating: experiment_01.csv       
  inflating: experiment_02.csv       
  inflating: experiment_03.csv       
  inflating: experiment_04.csv       
  inflating: experiment_05.csv       
  inflating: experiment_06.csv       
  inflating: experiment_07.csv       
  inflating: experiment_08.csv       
  inflating: experiment_09.csv       
  inflating: experiment_10.csv       
  inflating: experiment_11.csv       
  inflating: experiment_12.csv       
  inflating: experiment_13.csv       
  inflating: experiment_14.csv       
  inflating: experiment_15.csv       
  inflating: experiment_16.csv       
  inflating: experiment_17.csv       
  inflating: experiment_18.csv       
  inflating: test_artifact.jpg       
  inflating: train.csv               


In [5]:
base_path = '/content/'

In [6]:
outcomes = pd.read_csv(base_path + 'train.csv')

In [7]:
part_out = outcomes[outcomes['passed_visual_inspection'].notna()]
pprint(pd.crosstab(part_out.tool_condition, part_out.passed_visual_inspection))

passed = part_out.passed_visual_inspection.eq('yes').mul(1)
wear = part_out.tool_condition.eq('unworn').mul(1)

print('\nPearson correlation coefficient: {:.2f}'.format(matthews_corrcoef(passed, wear)))

passed_visual_inspection  no  yes
tool_condition                   
unworn                     0    6
worn                       4    4

Pearson correlation coefficient: 0.55


This correlation coefficient value can be interpreted as a loose correlation between a tool being unworn and a part passing visual inspection.

In [8]:
experiment1 = pd.read_csv(base_path + 'experiment_01.csv')
experiment1.reset_index(inplace=True)

In [9]:
experiment1.head(3)

,index,X1_ActualPosition,X1_ActualVelocity,X1_ActualAcceleration,X1_CommandPosition,X1_CommandVelocity,X1_CommandAcceleration,X1_CurrentFeedback,X1_DCBusVoltage,X1_OutputCurrent,...,S1_CurrentFeedback,S1_DCBusVoltage,S1_OutputCurrent,S1_OutputVoltage,S1_OutputPower,S1_SystemInertia,M1_CURRENT_PROGRAM_NUMBER,M1_sequence_number,M1_CURRENT_FEEDRATE,Machining_Process
0,0,198.0,0.0,0.00,198.0,0.0,0.000000,0.18,0.0207,329.0,...,0.524,2.740000e-19,329.0,0.0,6.960000e-07,12.0,1.0,0.0,50.0,Starting
1,1,198.0,-10.8,-350.00,198.0,-13.6,-358.000000,-10.90,0.1860,328.0,...,-0.288,2.740000e-19,328.0,0.0,-5.270000e-07,12.0,1.0,4.0,50.0,Prep
2,2,196.0,-17.8,-6.25,196.0,-17.9,-0.000095,-8.59,0.1400,328.0,...,0.524,2.740000e-19,328.0,0.0,9.100000e-07,12.0,1.0,7.0,50.0,Prep


In [10]:
experiment1[['Machining_Process', 'index']].groupby('Machining_Process').count()

,index
Machining_Process,
Layer 1 Down,148
Layer 1 Up,172
Layer 2 Down,132
Layer 2 Up,203
Layer 3 Down,142
Layer 3 Up,194
Prep,30
Repositioning,25
Starting,1


Assumption is that the only data that contains useful information for detecting CNC cutting health is the data that represents active cutting actions by the CNC. Other actions - start-up, repositioning, end - will be consistent across experiments regardless of the health of the cutting tool.

# Define functions

In [11]:
# Prep data functions
def clean_data(data):
    """Use this function to keep only CNC active cutting actions"""
    keep_act = ['Layer 1 Down', 'Layer 1 Up', 'Layer 2 Down', 'Layer 2 Up', 'Layer 3 Down', 'Layer 3 Up']
    data = data[data['Machining_Process'].isin(keep_act)]
#     print(data[['Machining_Process', 'index']].groupby('Machining_Process').count())
    
    data.drop('Machining_Process', inplace=True, axis=1)
    return data

def scale_decompose(data, pca_n=3):
    # Scale data
    scaler = MinMaxScaler(feature_range=(0, 1))
    scale_exper = scaler.fit_transform(data)

    # Apply PCA to data
    pca = PCA(n_components=pca_n, svd_solver='full')
    return pca.fit_transform(scale_exper)

# Calculate Mahalanobis dist functions
def is_pos_def(A):
    if np.allclose(A, A.T):
        try:
            np.linalg.cholesky(A)
            return True
        except np.linalg.LinAlgError:
            return False
    else:
        return False
    
def cov_matrix(data, verbose=False):
    covariance_matrix = np.cov(data, rowvar=False)
    if is_pos_def(covariance_matrix):
        inv_covariance_matrix = np.linalg.inv(covariance_matrix)
        if is_pos_def(inv_covariance_matrix):
            return covariance_matrix, inv_covariance_matrix
        else:
            print("Error: Inverse of Covariance Matrix is not positive definite!")
    else:
        print("Error: Covariance Matrix is not positive definite!")
        
def MahalanobisDist(inv_cov_matrix, mean_distr, data, verbose=False):
    diff = data - mean_distr
    md = []
    for i in range(len(diff)):
        md.append(np.sqrt(diff[i].dot(inv_cov_matrix).dot(diff[i])))
    return md

def MD_detectOutliers(dist, extreme=False, verbose=False):
    k = 3. if extreme else 2.
    threshold = np.mean(dist) * k
    outliers = []
    for i in range(len(dist)):
        if dist[i] >= threshold:
            outliers.append(i)  # index of the outlier
    return np.array(outliers)

def MD_threshold(dist, extreme=False, verbose=False):
    k = 3. if extreme else 2.
    threshold = np.mean(dist) * k
    return threshold

In [12]:
def full_process(experiment_n, components=2, chi2_print=True, exper_num=None):
    """Experiment data should only contain the columns that are desireable"""
    exper_pca = scale_decompose(experiment_n, pca_n=components)

    cov, inv_cov = cov_matrix(exper_pca)
    mean_dist = exper_pca.mean(axis=0)

    m_dist = MahalanobisDist(inv_cov, mean_dist, exper_pca)
    
    if chi2_print:
        fig_x = go.Figure(
            data=[go.Histogram(x=np.square(m_dist))],
            layout=go.Layout({'title': 'X^2 Distribution'})
        )
        fig_x.show()
    
    if exper_num:
        title = 'Mahalanobis Distribution Experiment {}'.format(exper_num)
    else:
        title = 'Mahalanobis Distribution'
    fig_m = pff.create_distplot([m_dist], group_labels=['m_dist'], bin_size=0.15)
    fig_m.update_layout(title_text=title)
    fig_m.show()
    
    return exper_pca, m_dist

In [13]:
!pip install -U kaleido

     |████████████████████████████████| 79.9 MB 125 kB/s 


In [14]:
def corr_actualcommand(data, corr_cols):
    look_data = data[corr_cols]
    corr_data = look_data.corr()

    fig = go.Figure(data=go.Heatmap(
        z=corr_data.values,
        x=list(corr_data.index),
        y=list(corr_data.columns)
    ))
    fig.show()
    #fig.show(renderer="png")

# Feature engineer
Bad practice for data science but I'm going to look at a single unworn tool experiment and a single worn tool experiment to make some generalizations about the data. Mostly looking to see that they follow the same correlations when it comes to COMMAND and ACTUAL positioning data.

My hypothesis is that the COMMAND feature and its corresponding ACTUAL feature will both be highly correlated and represent largely the exact same value, and that these columns do not actually provide much information when it comes to detecting tool wear quality.

In [15]:
# Unworn data
# Dataset loaded above
# Clean up data
exper1_clean = clean_data(experiment1)
clean_ex1 = exper1_clean[(exper1_clean['M1_CURRENT_FEEDRATE']!=50) & (exper1_clean['X1_ActualPosition']!=198)]
print('')
print('Length of data before inaccurate points removed {:d}'.format(len(exper1_clean)))
print('Length of data after inaccurate points removed {:d}'.format(len(clean_ex1)))


Length of data before inaccurate points removed 991
Length of data after inaccurate points removed 991


Give the length of data before and after the described (in dataset description) inaccurate points, it is safe to conclude that experiment 1 produced a clean dataset that did not contain false signals.

In [16]:
import plotly.io as pio
pio.renderers.default = "colab"

In [17]:
columns = exper1_clean.columns
corr_cols = list(filter(lambda x: ('Actual' in x) | ('Command' in x), columns))
corr_actualcommand(exper1_clean, corr_cols)

In [18]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=exper1_clean['index'], y=exper1_clean['X1_ActualPosition'],
    mode='lines',
    name='x-actual'
))
fig.add_trace(go.Scatter(
    x=exper1_clean['index'], y=exper1_clean['Y1_ActualPosition'],
    mode='lines',
    name='y-actual'
))

fig

In [22]:
outcomes.head(20)

,No,material,feedrate,clamp_pressure,tool_condition,machining_finalized,passed_visual_inspection
0,1,wax,6,4.0,unworn,yes,yes
1,2,wax,20,4.0,unworn,yes,yes
2,3,wax,6,3.0,unworn,yes,yes
3,4,wax,6,2.5,unworn,no,NaN
4,5,wax,20,3.0,unworn,no,NaN
5,6,wax,6,4.0,worn,yes,no
6,7,wax,20,4.0,worn,no,NaN
7,8,wax,20,4.0,worn,yes,no
8,9,wax,15,4.0,worn,yes,no
9,10,wax,12,4.0,worn,yes,no


In [21]:
# Worn data
# Load data
experiment2 = pd.read_csv(base_path + 'experiment_02.csv')
experiment2.reset_index(inplace=True)

# Clean up data
exper2_clean = clean_data(experiment2)
clean_ex2 = exper2_clean[(exper2_clean['M1_CURRENT_FEEDRATE']!=50) & (exper2_clean['X1_ActualPosition']!=198)]
print('')
print('Length of data before inaccurate points removed {:d}'.format(len(exper2_clean)))
print('Length of data after inaccurate points removed {:d}'.format(len(clean_ex2)))


Length of data before inaccurate points removed 1069
Length of data after inaccurate points removed 210


In [ ]:
# Worn data
# Load data
experiment2 = pd.read_csv(base_path + 'experiment_02.csv')
experiment2.reset_index(inplace=True)

# Clean up data
exper2_clean = clean_data(experiment2)
clean_ex2 = exper2_clean[(exper2_clean['M1_CURRENT_FEEDRATE']!=50) & (exper2_clean['X1_ActualPosition']!=198)]
print('')
print('Length of data before inaccurate points removed {:d}'.format(len(exper2_clean)))
print('Length of data after inaccurate points removed {:d}'.format(len(clean_ex2)))

In [20]:
# Worn data
# Load data
experiment6 = pd.read_csv(base_path + 'experiment_06.csv')
experiment6.reset_index(inplace=True)

# Clean up data
exper6_clean = clean_data(experiment6)
clean_ex6 = exper6_clean[(exper6_clean['M1_CURRENT_FEEDRATE']!=50) & (exper6_clean['X1_ActualPosition']!=198)]
print('')
print('Length of data before inaccurate points removed {:d}'.format(len(exper6_clean)))
print('Length of data after inaccurate points removed {:d}'.format(len(clean_ex6)))


Length of data before inaccurate points removed 1036
Length of data after inaccurate points removed 853


In [23]:
columns = exper6_clean.columns
corr_cols = list(filter(lambda x: ('Actual' in x) | ('Command' in x), columns))
corr_actualcommand(exper6_clean, corr_cols)

In [24]:
fig = make_subplots(rows=3, cols=1)
fig.add_trace(
    go.Scatter(
        x=exper6_clean['index'], y=exper6_clean['X1_ActualPosition'],
        mode='lines',
        name='x-actual'
    ),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(
        x=exper6_clean['index'], y=exper6_clean['Y1_ActualPosition'],
        mode='lines',
        name='y-actual'
    ),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(
        x=exper6_clean['index'], y=exper6_clean['X1_ActualPosition'],
        mode='lines',
        name='x-actual'
    ),
    row=2, col=1
)
fig.add_trace(
    go.Scatter(
        x=exper6_clean['index'], y=exper6_clean['Z1_ActualPosition'],
        mode='lines',
        name='z-actual'
    ),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(
        x=exper6_clean['index'], y=exper6_clean['X1_ActualPosition'],
        mode='lines',
        name='x-actual'
    ),
    row=3, col=1
)
fig.add_trace(
    go.Scatter(
        x=exper6_clean['index'], y=exper6_clean['S1_ActualVelocity'],
        mode='lines',
        name='s-actual'
    ),
    row=3, col=1
)

fig.show()

It appears that the bad signal present in the data is causing correlation between features that should not show correlation - e.g. X1_ActualPosition and Y1_ActualPosition or X1_ActualPosition and Z1_ActualPosition. The line plots allow visual validation that the bad readings represented by X1_ActualPosition = 198 are cross contaminating the signal from the other sensors.

Now I am left to question whether these false readings in the command and actual positions are bleeding into the raw current, voltage, and power signals. This is vital because my hypothesis is that these signals provide better signal with which to detect the tool wear.

In [25]:
columns = exper6_clean.columns
raw_cols = list(filter(lambda x: ('Current' in x) | ('Voltage' in x) | ('Power' in x), columns))
corr_actualcommand(exper6_clean, raw_cols+['X1_ActualPosition'])

In [26]:
fig = make_subplots(rows=3, cols=1)
fig.add_trace(
    go.Scatter(
        x=exper6_clean['index'], y=exper6_clean['X1_ActualPosition'],
        mode='lines',
        name='x-actual'
    ),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(
        x=exper6_clean['index'], y=exper6_clean['S1_OutputVoltage'],
        mode='lines',
        name='s-volt'
    ),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(
        x=exper6_clean['index'], y=exper6_clean['X1_ActualPosition'],
        mode='lines',
        name='x-actual'
    ),
    row=2, col=1
)
fig.add_trace(
    go.Scatter(
        x=exper6_clean['index'], y=exper6_clean['S1_OutputPower']*100,
        mode='lines',
        name='s-power'
    ),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(
        x=exper6_clean['index'], y=exper6_clean['X1_ActualPosition'],
        mode='lines',
        name='x-actual'
    ),
    row=3, col=1
)
fig.add_trace(
    go.Scatter(
        x=exper6_clean['index'], y=exper6_clean['S1_DCBusVoltage']*100,
        mode='lines',
        name='s-bus'
    ),
    row=3, col=1
)

fig.show()

Based on visual inspection above and the fact that the correlation coefficients are not much larger than +/- 0.5, I may be able to keep the signal during the inaccurate X1_ActualPosition readings in the analysis rather than cutting it out when I keep only the raw signals rather than the position/velocity/acceleration signals.

In [27]:
# Collect columns that are desireable in further analysis
keeper_cols = list(filter(lambda x: 'Z1' not in x, raw_cols))

# Define number of PCA components
component = 2 # should use 2 because this fits the basic definition of the Mahalanobis distance

In [28]:
# Perform outlier analysis
exper1_pca, exper1_mdist = full_process(exper1_clean[keeper_cols], components=component)
thresh = MD_threshold(exper1_mdist)

In [29]:
# Perform outlier analysis
exper6_pca, exper6_mdist = full_process(exper6_clean[keeper_cols], components=component)

In [31]:
# Load another dataset - worn
experiment8 = pd.read_csv(base_path + 'experiment_08.csv')
experiment8.reset_index(inplace=True)

# Clean up data
exper8_clean = clean_data(experiment8)
clean_ex8 = exper8_clean[(exper8_clean['M1_CURRENT_FEEDRATE']!=50) & (exper8_clean['X1_ActualPosition']!=198)]
print('')
print('Length of data before inaccurate points removed {:d}'.format(len(exper8_clean)))
print('Length of data after inaccurate points removed {:d}'.format(len(clean_ex8)))


Length of data before inaccurate points removed 412
Length of data after inaccurate points removed 259


In [32]:
# Perform outlier analysis
exper8_pca, exper8_mdist = full_process(exper8_clean[keeper_cols], components=component)

In [34]:
# Load another dataset - unworn
experiment3 = pd.read_csv(base_path + 'experiment_03.csv')
experiment3.reset_index(inplace=True)

# Clean up data
exper3_clean = clean_data(experiment3)
clean_ex3 = exper3_clean[(exper3_clean['M1_CURRENT_FEEDRATE']!=50) & (exper3_clean['X1_ActualPosition']!=198)]
print('')
print('Length of data before inaccurate points removed {:d}'.format(len(exper3_clean)))
print('Length of data after inaccurate points removed {:d}'.format(len(clean_ex3)))


Length of data before inaccurate points removed 1072
Length of data after inaccurate points removed 726


In [35]:
# Perform outlier analysis
exper3_pca, exper3_mdist = full_process(exper3_clean[keeper_cols], components=component)

In [36]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=exper1_clean.reset_index().index, y=exper1_mdist,
    mode='lines',
    name='unworn_01'
))
fig.add_trace(go.Scatter(
    x=exper6_clean.reset_index().index, y=exper6_mdist,
    mode='lines',
    name='worn_06'
))
fig.add_trace(go.Scatter(
    x=exper8_clean.reset_index().index, y=exper8_mdist,
    mode='lines',
    name='worn_08'
))
fig.add_trace(go.Scatter(
    x=exper3_clean.reset_index().index, y=exper3_mdist,
    mode='lines',
    name='unworn_03'
))
fig.add_shape(
    type='line',
    y0=thresh,
    y1=thresh,
    x0=0,
    x1=max([len(exper1_mdist), len(exper6_mdist)]),
    line=dict(color='RoyalBlue', width=2, dash='dot')
)
fig.update_shapes(dict(xref='x', yref='y'))
fig.show()

This analysis is not turning out to be effect. "Worn" tools fall within the Mahalanobis distance threshold more frequently than the "unworn" tools. I can't determine why. Thinking it could be the wrong technique or it could be the noise in the data overwhelming the signal.

# Last ditch effort
Combine all of the unworn datasets into one and then calculate the Mahalanobis distance from the combined dataset.

In [38]:
completed_exper = outcomes[outcomes['machining_finalized']=='yes']

unworn = []
idx_unworn = []
worn = []
idx_worn = []
for i, r in completed_exper.iterrows():
    if r['tool_condition'] == 'unworn':
        if r['No'] < 10:
            unw_data = pd.read_csv(base_path + 'experiment_0{}.csv'.format(r['No']))
        else:
            unw_data = pd.read_csv(base_path + 'experiment_{}.csv'.format(r['No']))
        unw_data['Experiment'] = r['No']
        unworn.append(unw_data)
        idx_unworn.append(r['No'])
    elif r['tool_condition'] == 'worn':
        if r['No'] < 10:
            w_data = pd.read_csv(base_path + 'experiment_0{}.csv'.format(r['No']))
        else:
            w_data = pd.read_csv(base_path + 'experiment_{}.csv'.format(r['No']))
        w_data['Experiment'] = r['No']
        worn.append(w_data)
        idx_worn.append(r['No'])
    
unworn_df = pd.concat(unworn, ignore_index=True)
worn_df = pd.concat(worn, ignore_index=True)

In [39]:
unworn_clean = clean_data(unworn_df)
worn_clean = clean_data(worn_df)

reduce_unworn = unworn_clean[(unworn_clean['M1_CURRENT_FEEDRATE']!=50) & (unworn_clean['X1_ActualPosition']!=198)]
reduce_worn = worn_clean[(worn_clean['M1_CURRENT_FEEDRATE']!=50) & (worn_clean['X1_ActualPosition']!=198)]

print('Unworn data')
for ix in idx_unworn:
    print('% data with noise experiment {}: {:.2f}'.format(
        ix,
        (1
         - len(reduce_unworn[reduce_unworn['Experiment']==ix])
         / len(unworn_clean[unworn_clean['Experiment']==ix]))
    ))

print('Worn data')
for ix in idx_worn:
    print('% data with noise experiment {}: {:.2f}'.format(
        ix,
        (1
         - len(reduce_worn[reduce_worn['Experiment']==ix])
         / len(worn_clean[worn_clean['Experiment']==ix]))
    ))

Unworn data
% data with noise experiment 1: 0.00
% data with noise experiment 2: 0.80
% data with noise experiment 3: 0.32
% data with noise experiment 11: 0.16
% data with noise experiment 12: 0.11
% data with noise experiment 17: 0.11
Worn data
% data with noise experiment 6: 0.18
% data with noise experiment 8: 0.37
% data with noise experiment 9: 0.38
% data with noise experiment 10: 0.59
% data with noise experiment 13: 0.16
% data with noise experiment 14: 0.16
% data with noise experiment 15: 0.32
% data with noise experiment 18: 0.15


Based on the amount of bad signal from above, going to eliminate experiment 2 as dead on arrival. There is just too much bad signal in this experiment to make it useful. 

In [40]:
# Remove bad experiment
unworn_clean = unworn_clean[unworn_clean['Experiment']!=2]

# Perform outlier analysis
unworn_pca, unworn_mdist = full_process(unworn_clean[keeper_cols], components=component)

thresh = MD_threshold(unworn_mdist)
print('Threshold: {:0.2f}'.format(thresh))

Threshold: 2.08


In [43]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=list(range(0, len(unworn_mdist))),
    y=unworn_mdist,
    mode='lines',
    name='unworn'
))

fig.add_shape(
    type='line',
    y0=thresh,
    y1=thresh,
    x0=0,
    x1=len(unworn_mdist),
    line=dict(color='RoyalBlue', width=2, dash='dot')
)
fig.update_shapes(dict(xref='x', yref='y'))
fig.update_layout(title_text='Mahalanobis Distance Trace All Unworn Data')
fig.show()

In [42]:
unworn_test = pd.DataFrame(unworn_pca)

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=unworn_test[0],
    y=unworn_test[1],
    mode='markers',
    name='unworn'
))

fig.update_layout(title_text='PCA Plot')
fig.show()

## **Conclusion:**
Based on the Mahalanobis distance trace of the unworn data and the PCA plot, it appears that this technique is not appropriate for the dataset at hand. The reason for this is not completely clear. It could be the level of bad/contaminated signal in the dataset. It could be the bimodal nature of the distribution. It could be the level of bad signal in the data is affecting the PCA decomposition and causing the bimodal distribution. It could simply be that the worn tools do not generate signal picked up in the positioning sensors of the CNC that identifies them as outliers compared to unworn tools.

One last thing that could be checked is whether the over threshold Mahalanobis distance points correlate to the bad signal data points. However, I am not performing this analysis.

For completeness below is the analysis applied to each worn experiment.

In [44]:
# Perform outlier analysis
worn_pca = dict()
worn_mdist = dict()
for ix in idx_worn:
    worn_pca_n, worn_mdist_n = full_process(
        worn_clean[worn_clean['Experiment']==ix][keeper_cols],
        components=component,
        chi2_print=False,
        exper_num=ix
    )
    worn_pca[ix] = worn_pca_n
    worn_mdist[ix] = worn_mdist_n

In [45]:
fig = go.Figure()

x_size = []
for key in worn_mdist:
    x_size.append(len(worn_mdist[key]))
    fig.add_trace(go.Scatter(
        x=list(range(0, len(worn_mdist[key]))),
        y=worn_mdist[key],
        mode='lines',
        name='exper_{}'.format(key)
    ))

fig.add_shape(
    type='line',
    y0=thresh,
    y1=thresh,
    x0=0,
    x1=max(x_size),
    line=dict(color='RoyalBlue', width=2, dash='dot')
)
fig.update_shapes(dict(xref='x', yref='y'))
fig.update_layout(title_text='Mahalanobis Distance Trace of Each Worn Experiment')
fig.show()

In [46]:
fig = go.Figure()

for key in worn_pca:
    worn_test = pd.DataFrame(worn_pca[key])

    fig.add_trace(go.Scatter(
        x=worn_test[0],
        y=worn_test[1],
        mode='markers',
        name='exper_{}'.format(key)
    ))
    
fig.update_layout(title_text='PCA Plot')
fig.show()